In [6]:
import common_utils
import os
import pandas as pd

# Example usage
root_folder = '../../../../data/minimized_run_5'
filename = 'worker1.feather'
subfolders = common_utils.find_subfolders_with_file(root_folder, filename)
print(subfolders)
prom_data_paths = {os.path.basename(x): x for x in subfolders}
yolo_data_paths = {key: os.path.join(val, "yolo_qos.feather") for key, val in prom_data_paths.items()}


['../../../../data/minimized_run_5/1732144183_yolov8m_640', '../../../../data/minimized_run_5/1732123252_yolo11m_320', '../../../../data/minimized_run_5/1732156444_yolov8n_1280', '../../../../data/minimized_run_5/1732124777_yolov10s_320', '../../../../data/minimized_run_5/1732120270_yolov9m_160', '../../../../data/minimized_run_5/1732129367_yolov8l_320', '../../../../data/minimized_run_5/1732159264_yolov8x_1280', '../../../../data/minimized_run_5/1732129943_yolov8x_320', '../../../../data/minimized_run_5/1732142717_yolov8n_640', '../../../../data/minimized_run_5/1732151684_yolov10m_1280', '../../../../data/minimized_run_5/1732154682_yolov9m_1280', '../../../../data/minimized_run_5/1732149252_yolo11l_1280', '../../../../data/minimized_run_5/1732118879_yolov10m_160', '../../../../data/minimized_run_5/1732121677_yolov8m_160', '../../../../data/minimized_run_5/1732143492_yolov8s_640', '../../../../data/minimized_run_5/1732135028_yolov10s_640', '../../../../data/minimized_run_5/1732127347_y

In [7]:
from utils.header_cleaner import *
import difflib
import os


"""
Fetch paths to the data
"""
prom_data_paths = {os.path.basename(x): x for x in subfolders}
yolo_data_paths = {key: os.path.join(val, "yolo_qos.feather") for key, val in prom_data_paths.items()}

"""
Get corresponding yolo stats for each model 
"""
response_time = {}
for key in prom_data_paths.keys():
    yolo_df = common_utils.read_feather_cached(yolo_data_paths[key])
    yolo_df['total_inference_time'] = yolo_df['inf'] + yolo_df['post'] + yolo_df['pre']
    yolo_df['end_to_end_response_time'] = yolo_df['total_inference_time'] + yolo_df['queue']
    yolo_df['start'] = pd.to_datetime(yolo_df['start_time'], unit='ms')  # Convert to datetime (optional)
    yolo_df.set_index('start', inplace=True)
    resampled_df = yolo_df.resample('5S')
    model_info = common_utils.path_to_name_and_resolution(key)
    if model_info.resolution not in response_time:
        response_time[model_info.resolution] = {}
    response_time[model_info.resolution][model_info.model] = resampled_df.agg({'end_to_end_response_time': 'mean'}).reset_index()['end_to_end_response_time'].rename(key)


In [8]:
import plotly.express as px
from matplotlib import pyplot as plt

# Function to compute simple moving average
def moving_average(data, window_size):
    return data.rolling(window=window_size).mean()

# You can adjust the window size for different levels of smoothing
window_size = 3

for resolution in sorted(response_time.keys()):
    queue_df = pd.DataFrame.from_dict(response_time[resolution])
    fig = px.line(queue_df, x=queue_df.index, y=queue_df.columns)
    fig.update_layout(title=f'End-to-end latency (Resolution: {resolution})', xaxis_title='Time', yaxis_title='Reponse Time in MS', yaxis_type='log')
    fig.show()

In [9]:
import os
import pandas as pd
from functools import lru_cache
from collections import namedtuple
import difflib
import plotly.express as px

response_time = {}
for key in prom_data_paths.keys():
    yolo_df = common_utils.read_feather_cached(yolo_data_paths[key])
    yolo_df['total_inference_time'] = yolo_df['inf'] + yolo_df['post'] + yolo_df['pre']
    yolo_df['end_to_end_response_time'] = yolo_df['total_inference_time'] + yolo_df['queue']
    yolo_df['start'] = pd.to_datetime(yolo_df['start_time'], unit='ms')
    yolo_df.set_index('start', inplace=True)
    resampled_df = yolo_df.resample('5S')
    model_info = common_utils.path_to_name_and_resolution(key)
    if model_info.resolution not in response_time:
        response_time[model_info.resolution] = {}
    response_time[model_info.resolution][model_info.model] = resampled_df.agg({'end_to_end_response_time': 'min'}).reset_index()['end_to_end_response_time'].rename(key)

data = []
for resolution, models in response_time.items():
    for model, latency in models.items():
        if not latency.empty:
            min_latency = latency.min()
            data.append((str(resolution), model, min_latency))

# Sort data before creating the DataFrame
sorted_data = sorted(data, key=lambda x: common_utils._sort_key_size_version(x[1]))

df = pd.DataFrame(sorted_data, columns=['Resolution', 'Model', 'Min Latency'])
fig = px.bar(df, x='Model', y='Min Latency', color='Resolution', barmode="group", title=f'Minimum end-to-end latency', log_y=True)
fig.update_layout(yaxis_title='Min Latency (ms)')
fig.show()

In [13]:
df.to_csv('table_plots/data_min_latency.csv')

In [10]:
import os
import pandas as pd
from functools import lru_cache
from collections import namedtuple
import difflib
import plotly.express as px

response_time = {}
for key in prom_data_paths.keys():
    yolo_df = common_utils.read_feather_cached(yolo_data_paths[key])
    yolo_df['total_inference_time'] = yolo_df['inf'] + yolo_df['post'] + yolo_df['pre']
    yolo_df['end_to_end_response_time'] = yolo_df['total_inference_time'] + yolo_df['queue']
    yolo_df['start'] = pd.to_datetime(yolo_df['start_time'], unit='ms')
    yolo_df.set_index('start', inplace=True)
    resampled_df = yolo_df.resample('5S')
    model_info = common_utils.path_to_name_and_resolution(key)
    if model_info.resolution not in response_time:
        response_time[model_info.resolution] = {}
    response_time[model_info.resolution][model_info.model] = resampled_df.agg({'end_to_end_response_time': 'min'}).reset_index()['end_to_end_response_time'].rename(key)

data = []

# Sorting resolutions to ensure we are correctly calculating differences from previous ones
sorted_resolutions = sorted(response_time.keys())

for resolution in sorted_resolutions:
    for model, latency in response_time[resolution].items():
        if not latency.empty:
            if sorted_resolutions.index(resolution) > 0:
                previous_resolution = sorted_resolutions[sorted_resolutions.index(resolution) - 1]
                if model in response_time[previous_resolution]:
                    previous_latency = response_time[previous_resolution][model]
                    if not previous_latency.empty:
                        min_latency = latency.min() - previous_latency.min()
                        data.append((str(resolution), model, min_latency))
                else:
                    min_latency = latency.min()
                    data.append((str(resolution), model, min_latency))
            else:
                min_latency = latency.min()
                data.append((str(resolution), model, min_latency))

df = pd.DataFrame(data, columns=['Resolution', 'Model', 'Min Latency'])
fig = px.bar(df, x='Model', y='Min Latency', color='Resolution', barmode="stack", title=f'Minimum end-to-end latency (overlapping bars)', log_y=True)
fig.show()
    
    
"""
Same as above, but here the bar peaks match to actual latency values, instead of cumulative latency over all resolutions
"""

'\nSame as above, but here the bar peaks match to actual latency values, instead of cumulative latency over all resolutions\n'